In [1]:
import os
import math
import random
import implicit
import datetime
import pandas as pd
import numpy as np
import scipy.sparse as sparse

from cdhf.data import Data
from implicit import evaluation

from tqdm.notebook import tqdm as log_progress

random.seed(42)
random_sample = random.sample(range(10, 300), 1)
data = Data("../input/mmdata.json")
data.load_all()

In [2]:
df = pd.DataFrame.from_records([vars(cm) for cm in data.channel_members])
df["index"] = df["channel_id"] + "-" + df["user_id"]
df.set_index('index', inplace=True)

df_grouped_users = df.groupby(["channel_id"]).count()
allowed_channels = df_grouped_users[df_grouped_users["user_id"] > 5].index.array
df = df[df["channel_id"].isin(allowed_channels)]
df['u_id'] = df['user_id'].astype("category").cat.codes
df['c_id'] = df['channel_id'].astype("category").cat.codes

In [3]:
class MeasureOfConfidence():
    def calculate(self):
        pass

    def name(self):
        pass

    def preprocess_cui(self, cui, df):
        pass

class BinaryMeasureOfConfidence(MeasureOfConfidence):
    def calculate(self, r_ui, alpha):
        return 1

    def name(self):
        return "Binary Measure < 1 >"

class SimpleMeasureOfConfidence(MeasureOfConfidence):
    def calculate(self, r_ui, alpha):
        return 1 + alpha*r_ui

    def name(self):
        return "Simple Measure < 1 + alpha*r_ui >"

class LogMeasureOfConfidence(MeasureOfConfidence):
    def calculate(self, r_ui, alpha):
        return 1 + alpha*(math.log(1+(r_ui/0.01)))

    def name(self):
        return "Log Measure < 1 + alpha*log(1 + r_ui/epsilon) >"

class EvalResults():

    def __init__(self, name, confidence, metrics, factors, alpha, iterations, feature):
        self.name = name
        self.confidence = confidence
        self.metrics = metrics
        self.factors = factors
        self.alpha = alpha
        self.iterations = iterations
        self.feature = feature

class BasePowerFunction():
    def name():
        pass

    def calculate():
        pass

class SimpleLScore(BasePowerFunction):
    def name(self):
        return "Simple Score MSG Count"

    def calculate(self, s_l, s_r):
        return s_l

class SimpleRScore(BasePowerFunction):
    def name(self):
        return "Simple Score Feature"

    def calculate(self, s_l, s_r):
        return s_r

class PowerFuncScore(BasePowerFunction):
    def name(self):
        return "MSG COUNT To The Power of Score Feature"

    def calculate(self, s_l, s_r):
        return s_l.pow(1 + s_r)

power_functions = [SimpleLScore(), PowerFuncScore()]


def calculateSparsity(df):
    # Create lists of all users, artists and plays
    users = list(np.sort(df.u_id.unique()))
    channels = list(np.sort(df.c_id.unique()))
    interactions = list(df.score)

    sparsity = (1 - (len(interactions)/(len(users)*len(channels))))*100

    user_sparsity_df = df.copy()
    user_sparsity_df = user_sparsity_df.groupby(["u_id"]).count()
    user_sparsity_max = user_sparsity_df.c_id.max()
    user_sparsity_df['USS'] = (1 - (user_sparsity_df["c_id"]/user_sparsity_max)) * 100

    item_sparsity_df = df.copy()
    item_sparsity_df = item_sparsity_df.groupby(["c_id"]).count()
    item_rating_max = item_sparsity_df.u_id.max()
    item_sparsity_df['ISS'] = (1 - (item_sparsity_df["u_id"]/item_rating_max)) * 100

    return sparsity, user_sparsity_df['USS'].mean(), item_sparsity_df['ISS'].mean(), len(users), len(channels)


In [4]:

def execute(df, feature, power_function_name):
    sparsity, USS, ISS, users, channels = calculateSparsity(df)
    confidence_measures = [SimpleMeasureOfConfidence(), LogMeasureOfConfidence(), BinaryMeasureOfConfidence()]
    alpha_values = [10, 40, 80, 150, 200]
    factor_options = [25,50,100,150,200,250,300]
    iteration_opttions =[25, 50, 100, 150] 

    evaluation_results = []

    for alpha_val in alpha_values:        
        for confidence_measure in confidence_measures:                        
            c_ui = df['score'].astype(float).copy()
            c_ui.apply(confidence_measure.calculate, args= (alpha_val,))
            
            sparse_item_user = sparse.csr_matrix(( c_ui, (df['c_id'].astype(int), df['u_id'].astype(int))))
            # sparse_user_item = sparse.csr_matrix(( c_ui, (df['u_id'], df['c_id'])))
            for randomInt in random_sample:
                data_train, data_test = evaluation.train_test_split(sparse_item_user, 0.2, randomInt)

                for factors in factor_options:                    
                    for iterations in iteration_opttions:                        
                        regularization = 0.1                
                        models = []        
                        
                        models.append(implicit.als.AlternatingLeastSquares(num_threads = 4, factors=factors, regularization=regularization, iterations=iterations))    
                        models.append(implicit.cpu.bpr.BayesianPersonalizedRanking(num_threads = 4, factors=factors, iterations=iterations))
                        models.append(implicit.cpu.lmf.LogisticMatrixFactorization(num_threads = 4))                    
                        models.append(implicit.nearest_neighbours.ItemItemRecommender(num_threads = 4, K=20))
            
                        for model in models:
                            model.fit(data_train, show_progress=True)                              
                            ranking = implicit.evaluation.ranking_metrics_at_k(model, data_train, data_test, K=10, show_progress=False)
                            evalResult = EvalResults(name = model.__class__, confidence = confidence_measure.name(), metrics = ranking, factors = factors, alpha = alpha_val, iterations = iterations, feature = feature )                    
                            evaluation_results.append(evalResult)                                                

    df_evals = pd.DataFrame.from_records([[er.name.__doc__.split("\n\n")[0], er.metrics["auc"], er.metrics["precision"], er.factors, er.iterations, er.alpha, er.confidence, er.feature, power_function_name, er.metrics["ndcg"], er.metrics["map"], sparsity, USS, ISS, users, channels] for er in evaluation_results], columns=["name",'AUC', 'precision', 'factors', 'iterations', 'alpha', 'confidence_measure', 'feature', 'power_function_name', 'ndcg', 'map', "sparsity", "USS", "ISS", "users", "channels"])
    df_evals = df_evals.sort_values(['AUC', 'precision', 'iterations', 'factors'], ascending = [False, False, True, True])    

    return df_evals  


In [5]:
base_dir = "Processed/"
feature_files = ["messag_factor.csv", "team_boost.csv", "channel_relevance_0_100.csv", "channel_relevance_100_0.csv", "channel_relevance_25_75.csv", "channel_relevance_5_5.csv", "channel_relevance_75_25.csv", "mention_factor.csv", "mention_message_25_75.csv", "mention_message_5_5.csv", "mention_message_75_25.csv"]

In [6]:

df_evals = pd.DataFrame()

feature_evals = {}

feature_path = "Processed/UserChannel"
feature_files = [f for f in os.listdir(feature_path) if os.path.isfile(os.path.join(feature_path, f))]

for feature_file in log_progress(feature_files):
    feature_evals[feature_file] = {}
    for power_function in power_functions:
        feature_evals[feature_file][power_function] = pd.DataFrame()
        feature_df = pd.read_pickle(os.path.join(feature_path, feature_file)).rename(columns={"userid":"user_id", "channelid":"channel_id"})
        feature_df["index"] = feature_df["channel_id"] + "-" + feature_df["user_id"]
        feature_df.set_index('index', inplace=True)        
        rec_df = pd.merge(df, feature_df, on=['user_id','channel_id'], how="outer").fillna(0)
        rec_df["score"] = power_function.calculate(rec_df["msg_count"], rec_df["score"])        
        feature_evals[feature_file][power_function] = execute(rec_df[["user_id", "channel_id", "score", "u_id", "c_id"]], feature_file, power_function.name())            
        break


for key in feature_evals:  
    for power_function in feature_evals[key]:
        eval_frame = feature_evals[key][power_function]
        eval_frame.to_pickle(f"{key}-{power_function}.pkl")
        display(f"{key}-{power_function}", eval_frame.describe())        

    

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/19492 [00:00<?, ?it/s]

...

,AUC,precision,factors,iterations,alpha,ndcg,map,sparsity,USS,ISS,users,channels
count,4.000000,4.000000,4.0,4.0,4.0,4.000000,4.000000,4.000000,4.000000,4.00000,4.0,4.0
mean,0.532295,0.101110,150.0,100.0,40.0,0.093622,0.059793,99.866243,99.907002,99.80943,19492.0,9510.0
std,0.042252,0.123481,0.0,0.0,0.0,0.121440,0.084171,0.000000,0.000000,0.00000,0.0,0.0
min,0.500794,0.005019,150.0,100.0,40.0,0.004300,0.001586,99.866243,99.907002,99.80943,19492.0,9510.0
25%,0.502221,0.020086,150.0,100.0,40.0,0.017655,0.009661,99.866243,99.907002,99.80943,19492.0,9510.0
50%,0.518539,0.061516,150.0,100.0,40.0,0.050356,0.027067,99.866243,99.907002,99.80943,19492.0,9510.0
75%,0.548613,0.142540,150.0,100.0,40.0,0.126323,0.077199,99.866243,99.907002,99.80943,19492.0,9510.0
max,0.591310,0.276390,150.0,100.0,40.0,0.269476,0.183451,99.866243,99.907002,99.80943,19492.0,9510.0


'user-channel-1-0.pkl-<__main__.SimpleLScore object at 0x7ff71bb4f7f0>'

,AUC,precision,factors,iterations,alpha,ndcg,map,sparsity,USS,ISS,users,channels
count,4.000000,4.000000,4.0,4.0,4.0,4.000000,4.000000,4.000000,4.000000,4.00000,4.0,4.0
mean,0.532128,0.100992,150.0,100.0,40.0,0.093526,0.059893,99.866243,99.907002,99.80943,19492.0,9510.0
std,0.042098,0.123487,0.0,0.0,0.0,0.121349,0.084057,0.000000,0.000000,0.00000,0.0,0.0
min,0.500817,0.004608,150.0,100.0,40.0,0.004086,0.001472,99.866243,99.907002,99.80943,19492.0,9510.0
25%,0.502040,0.020063,150.0,100.0,40.0,0.017670,0.010068,99.866243,99.907002,99.80943,19492.0,9510.0
50%,0.518414,0.061569,150.0,100.0,40.0,0.050401,0.027358,99.866243,99.907002,99.80943,19492.0,9510.0
75%,0.548503,0.142499,150.0,100.0,40.0,0.126258,0.077183,99.866243,99.907002,99.80943,19492.0,9510.0
max,0.590868,0.276222,150.0,100.0,40.0,0.269217,0.183385,99.866243,99.907002,99.80943,19492.0,9510.0


'user-channel-1-1.pkl-<__main__.SimpleLScore object at 0x7ff71bb4f7f0>'

,AUC,precision,factors,iterations,alpha,ndcg,map,sparsity,USS,ISS,users,channels
count,4.000000,4.000000,4.0,4.0,4.0,4.000000,4.000000,4.000000,4.000000,4.00000,4.0,4.0
mean,0.532240,0.100970,150.0,100.0,40.0,0.093115,0.059497,99.866243,99.907002,99.80943,19492.0,9510.0
std,0.042179,0.123613,0.0,0.0,0.0,0.121685,0.084373,0.000000,0.000000,0.00000,0.0,0.0
min,0.500912,0.004912,150.0,100.0,40.0,0.004316,0.001549,99.866243,99.907002,99.80943,19492.0,9510.0
25%,0.502126,0.019706,150.0,100.0,40.0,0.016298,0.008818,99.866243,99.907002,99.80943,19492.0,9510.0
50%,0.518456,0.061281,150.0,100.0,40.0,0.049448,0.026512,99.866243,99.907002,99.80943,19492.0,9510.0
75%,0.548569,0.142544,150.0,100.0,40.0,0.126265,0.077191,99.866243,99.907002,99.80943,19492.0,9510.0
max,0.591135,0.276405,150.0,100.0,40.0,0.269248,0.183417,99.866243,99.907002,99.80943,19492.0,9510.0
